In [51]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios, plot_summary)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 3개 파일을 불러왔습니다: 1201.csv, 401.csv, 89.csv
📊 데이터 Shape: I=3, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=773.00, M2=1199.00


In [52]:
only = gp.Model("only")
only.setParam("MIPGap", 1e-7)
only.setParam("OutputFlag", 0)

a = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="a")
bp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
g = only.addVars(I, T+1, S, vtype=GRB.CONTINUOUS, name="gamma")
gc = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="gamma_charge")
gd = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="gamma_discharge")
phi1 = only.addVars(I, T, S, vtype=GRB.BINARY, name="phi1")
phi2 = only.addVars(I, T, S, vtype=GRB.BINARY, name="phi2")
phi3 = only.addVars(I, T, S, vtype=GRB.BINARY, name="phi3")

only.update()

obj = gp.quicksum(P_DA[t] * a[i, t] for i, t in product(range(I), range(T))) \
    + gp.quicksum(1 / S * (P_RT[t, s] * bp[i, t, s] - P_PN[t] * bm[i, t, s]) for i, t, s in product(range(I), range(T), range(S)))

only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - a[i, t] == bp[i, t, s] - bm[i, t, s] + gc[i, t, s] - gd[i, t, s])
    only.addConstr(bp[i, t, s] <= R[i, t, s])
    only.addConstr(g[i, t + 1, s] == g[i, t, s] + gc[i, t, s] - gd[i, t, s])
    only.addConstr(gd[i, t, s] <= g[i, t, s])
    only.addConstr(gc[i, t, s] <= K[i] - g[i, t, s])
    only.addConstr(bp[i, t, s] <= M1 * phi3[i, t, s])
    only.addConstr(bm[i, t, s] <= M1 * (1 - phi3[i, t, s]))
    only.addConstr(bm[i, t, s] <= M1 * phi2[i, t, s])
    only.addConstr(gc[i, t, s] <= M1 * (1 - phi2[i, t, s]))
    only.addConstr(gc[i, t, s] <= M1 * phi1[i, t, s])
    only.addConstr(gd[i, t, s] <= M1 * (1 - phi1[i, t, s]))
for i, s in product(range(I), range(S)):
    only.addConstr(g[i, 0, s] == K0[i])

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {only.objVal}")
else:
    print("No optimal solution found.")

a_vals = np.array([[a[i, t].X for t in range(T)] for i in range(I)])
bp_vals = np.array([[[bp[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
bm_vals = np.array([[[bm[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
g_vals  = np.array([[[g[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
gc_vals = np.array([[[gc[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
gd_vals = np.array([[[gd[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])

Set parameter MIPGap to value 1e-07
Optimal solution found! Objective value: 782684.0262224367


In [53]:
set = gp.Model("set")
set.setParam("MIPGap", 1e-7)
set.setParam("OutputFlag", 0)

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
ep = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_plus")
em = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="e_minus")

yp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z = set.addVars(I, T + 1, S, vtype=GRB.CONTINUOUS, name="z")
zc = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")

p1 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p1")
p2 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p2")
p3 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p3")
p4 = set.addVars(I, T, S, vtype=GRB.BINARY, name="p4")

set.update()

obj = gp.quicksum(P_DA[t] * gp.quicksum(x[i, t] for i in range(I)) for t in range(T)) + gp.quicksum((1 / S) * (P_RT[t, s] * gp.quicksum(ep[i, t, s] for i in range(I)) - P_PN[t] * gp.quicksum(em[i, t, s] for i in range(I))) for t in range(T) for s in range(S))

set.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
    set.addConstr(yp[i, t, s] <= R[i, t, s])
    set.addConstr(zd[i, t, s] <= z[i, t, s])
    set.addConstr(zc[i, t, s] <= K[i] - z[i, t, s])
    set.addConstr(yp[i, t, s] <= M1 * p3[i, t, s])
    set.addConstr(ym[i, t, s] <= M1 * (1 - p3[i, t, s]))
    set.addConstr(ym[i, t, s] <= M1 * p2[i, t, s])
    set.addConstr(zc[i, t, s] <= M1 * (1 - p2[i, t, s]))
    set.addConstr(zc[i, t, s] <= M1 * p1[i, t, s])
    set.addConstr(zd[i, t, s] <= M1 * (1 - p1[i, t, s]))
    set.addConstr(z[i, t, s] <= K[i])
    set.addConstr(z[i, t + 1, s] == z[i, t, s] + zc[i, t, s] - zd[i, t, s])
for i, s in product(range(I), range(S)):
    set.addConstr(z[i, 0, s] == K0[i])

for i, t, s in product(range(I), range(T), range(S)):
    set.addConstr(ep[i, t, s] == yp[i, t, s] - gp.quicksum(d[i, j, t, s] for j in range(I)))
    set.addConstr(em[i, t, s] == ym[i, t, s] - gp.quicksum(d[j, i, t, s] for j in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= M2 * p4[i, t, s])
    set.addConstr(gp.quicksum(em[i, t, s] for i in range(I)) <= M2 * (1 - p4[i, t, s]))
    set.addConstr(d[i, i, t, s] == 0)

set.optimize()

if set.status == GRB.OPTIMAL:
    print(f"Optimal solution found! Objective value: {set.objVal}")
else:
    print("No optimal solution found.")
    
x_vals = np.array([[x[i, t].X for t in range(T)] for i in range(I)])
yp_vals = np.array([[[yp[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
ym_vals = np.array([[[ym[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
z_vals  = np.array([[[z[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
zc_vals = np.array([[[zc[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
zd_vals = np.array([[[zd[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
ep_vals = np.array([[[ep[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)]) 
em_vals = np.array([[[em[i, t, s].X for s in range(S)] for t in range(T)] for i in range(I)])
d_vals = np.array([[[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)] for i in range(I)])

Set parameter MIPGap to value 1e-07
Optimal solution found! Objective value: 789199.598950884


In [77]:
t, s = 14, 19

print(f"[Settlement Decomposition @ t={t}, s={s}]\n")
print(f"{'i':>3} | {'x·DA':>10} {'+y⁺·RT':>10} {'-y⁻·PN':>10} {'EXᵢ':>10} || {'given':>10} {'received':>10} {'INᵢ':>10} || {'EXᵢ+INᵢ':>10} || {'DISAGG':>10}")
print("-" * 123)

total_ext, total_int, total_sum = 0, 0, 0
check_ext, check_int, check_total = 0, 0, 0

for i in range(I):
    # External Settlement
    x_da = x[i, t].X * P_DA[t]
    yp_rt = yp[i, t, s].X * P_RT[t, s]
    ym_pn = ym[i, t, s].X * P_PN[t]
    ex_i = x_da + yp_rt - ym_pn

    # Internal Settlement
    d_ij = sum(d[i, j, t, s].X for j in range(I) if j != i)
    d_ji = sum(d[j, i, t, s].X for j in range(I) if j != i)

    given = d_ij * P_PN[t]
    received = d_ji * P_RT[t, s]
    in_i = given - received

    # Total per participant
    total_i = ex_i + in_i

    # Accumulate
    total_ext += ex_i
    total_int += in_i
    total_sum += total_i
    
    check_total = sum(x[i, t].X * P_DA[t] + ep[i, t, s].X * P_RT[t, s] - em[i, t, s].X * P_PN[t] for i in range(I))
    disagg = a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t]
    disagg_total = sum(a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t] for i in range(I))

    print(f"{i:>3} | {x_da:10.2f} {yp_rt:10.2f} {-ym_pn:10.2f} {ex_i:10.2f} || "
          f"{given:10.2f} {-received:10.2f} {in_i:10.2f} || {total_i:10.2f} || "
          f"{disagg:10.2f} ({total_i - disagg:5.1f})")

print("-" * 123)
print(f"{'Σ':>3} | {'':>10} {'':>10} {'':>10} {total_ext:10.2f} || "
      f"{'':>10} {'':>10} {total_int:10.2f} || {total_sum:10.2f} || "
      f"{disagg_total:10.2f}")

# print("-" * 123)
# print(f"{'⁋':>3} | {'':>10} {'':>10} {'':>10} {'':>10} || "
#       f"{'':>10} {'':>10} {'':>10} || {check_total:10.2f} || ")

#------------------------------------------------------------

print(f"\n[Settlement Decomposition @ t={t}, s={s}]\n")
print(f"{'i':>3} | {'x·DA':>10} {'+y⁺·RT':>10} {'-y⁻·PN':>10} {'EXᵢ':>10} || {'given':>10} {'received':>10} {'INᵢ':>10} || {'EXᵢ+INᵢ':>10} || {'DISAGG':>10}")
print("-" * 123)

total_ext, total_int, total_sum = 0, 0, 0
check_ext, check_int, check_total = 0, 0, 0

for i in range(I):
    # External Settlement
    x_da = x[i, t].X * P_DA[t]
    yp_rt = yp[i, t, s].X * P_RT[t, s]
    ym_pn = ym[i, t, s].X * P_PN[t]
    ex_i = x_da + yp_rt - ym_pn

    # Internal Settlement
    d_ij = sum(d[i, j, t, s].X for j in range(I) if j != i)
    d_ji = 0

    given = d_ij * (P_PN[t] - P_RT[t, s])
    received = 0
    in_i = given - received

    # Total per participant
    total_i = ex_i + in_i

    # Accumulate
    total_ext += ex_i
    total_int += in_i
    total_sum += total_i
    
    check_total = sum(x[i, t].X * P_DA[t] + ep[i, t, s].X * P_RT[t, s] - em[i, t, s].X * P_PN[t] for i in range(I))
    disagg = a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t]
    disagg_total = sum(a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t] for i in range(I))

    print(f"{i:>3} | {x_da:10.2f} {yp_rt:10.2f} {-ym_pn:10.2f} {ex_i:10.2f} || "
          f"{given:10.2f} {-received:10.2f} {in_i:10.2f} || {total_i:10.2f} || "
          f"{disagg:10.2f} ({total_i - disagg:5.1f})")

print("-" * 123)
print(f"{'Σ':>3} | {'':>10} {'':>10} {'':>10} {total_ext:10.2f} || "
      f"{'':>10} {'':>10} {total_int:10.2f} || {total_sum:10.2f} || "
      f"{disagg_total:10.2f}")

# print("-" * 123)
# print(f"{'⁋':>3} | {'':>10} {'':>10} {'':>10} {'':>10} || "
#       f"{'':>10} {'':>10} {'':>10} || {check_total:10.2f} || ")

#------------------------------------------------------------

print(f"\n[Settlement Decomposition @ t={t}, s={s}]\n")
print(f"{'i':>3} | {'x·DA':>10} {'+e⁺·RT':>10} {'-e⁻·PN':>10} {'received':>10} {'EXᵢ':>10} || {'given':>10} {'INᵢ':>10} || {'EXᵢ+INᵢ':>10} || {'DISAGG':>10}")
print("-" * 123)

total_ext, total_int, total_sum = 0, 0, 0
check_ext, check_int, check_total = 0, 0, 0

for i in range(I):
    # External Settlement
    x_da = x[i, t].X * P_DA[t]
    ep_rt = ep[i, t, s].X * P_RT[t, s]
    em_pn = em[i, t, s].X * P_PN[t]

    # Internal Settlement
    d_ij = sum(d[i, j, t, s].X for j in range(I) if j != i)
    d_ji = sum(d[j, i, t, s].X for j in range(I) if j != i)

    given = d_ij * (P_PN[t] - P_RT[t, s])
    received = d_ji * (P_PN[t] - P_RT[t, s])
    
    ex_i = x_da + ep_rt - em_pn - received
    in_i = given

    # Total per participant
    total_i = ex_i + in_i

    # Accumulate
    total_ext += ex_i
    total_int += in_i
    total_sum += total_i
    
    check_total = sum(x[i, t].X * P_DA[t] + ep[i, t, s].X * P_RT[t, s] - em[i, t, s].X * P_PN[t] for i in range(I))
    disagg = a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t]
    disagg_total = sum(a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t] for i in range(I))

    print(f"{i:>3} | {x_da:10.2f} {ep_rt:10.2f} {-em_pn:10.2f} {-received:10.2f} {ex_i:10.2f} || "
          f"{given:10.2f} {in_i:10.2f} || {total_i:10.2f} || "
          f"{disagg:10.2f} ({total_i - disagg:5.1f})")

print("-" * 123)
print(f"{'Σ':>3} | {'':>10} {'':>10} {'':>10} {'':>10} {total_ext:10.2f} || "
      f"{'':>10} {total_int:10.2f} || {total_sum:10.2f} || "
      f"{disagg_total:10.2f}")

# print("-" * 123)
# print(f"{'⁋':>3} | {'':>10} {'':>10} {'':>10} {'':>10} {'':>10} || "
#       f"{'':>10} {'':>10} || {check_total:10.2f} || ")


#------------------------------------------------------------

print(f"\n[Settlement Decomposition @ t={t}, s={s}]\n")
print(f"{'i':>3} | {'x·DA':>10} {'+e⁺·RT':>10} {'-e⁻·PN':>10} {'+marg':>10} {'-marg':>10} {'EXᵢ':>10} || {'given':>10} {'INᵢ':>10} || {'EXᵢ+INᵢ':>10} || {'DISAGG':>10}")
print("-" * 130)

total_ext, total_int, total_sum = 0, 0, 0
check_ext, check_int, check_total = 0, 0, 0

for i in range(I):
    # External Settlement
    x_da = x[i, t].X * P_DA[t]
    ep_rt = ep[i, t, s].X * P_RT[t, s]
    em_pn = em[i, t, s].X * P_PN[t]

    # Internal Settlement
    d_ij = sum(d[i, j, t, s].X for j in range(I) if j != i)
    d_ji = sum(d[j, i, t, s].X for j in range(I) if j != i)

    given = d_ij * (P_PN[t] - P_RT[t, s])
    marg_plus = d_ij * (P_RT[t, s] - P_DA[t])
    marg_minus = d_ji * (P_PN[t] - P_DA[t])
    
    ex_i = x_da + ep_rt - em_pn + marg_plus - marg_minus
    in_i = given

    # Total per participant
    total_i = ex_i + in_i

    # Accumulate
    total_ext += ex_i
    total_int += in_i
    total_sum += total_i
    
    check_total = sum(x[i, t].X * P_DA[t] + ep[i, t, s].X * P_RT[t, s] - em[i, t, s].X * P_PN[t] for i in range(I))
    disagg = a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t]
    disagg_total = sum(a[i, t].X * P_DA[t] + bp[i, t, s].X * P_RT[t, s] - bm[i, t, s].X * P_PN[t] for i in range(I))

    print(f"{i:>3} | {x_da:10.2f} {ep_rt:10.2f} {-em_pn:10.2f} {marg_plus:10.2f} {-marg_minus:10.2f} {ex_i:10.2f} || "
          f"{given:10.2f} {in_i:10.2f} || {total_i:10.2f} || "
          f"{disagg:10.2f} ({total_i - disagg:5.1f})")

print("-" * 130)
print(f"{'Σ':>3} | {'':>10} {'':>10} {'':>10} {'':>10} {'':>10} {total_ext:10.2f} || "
      f"{'':>10} {total_int:10.2f} || {total_sum:10.2f} || "
      f"{disagg_total:10.2f}")

print("-" * 130)
print(f"{'⁋':>3} | {'':>10} {'':>10} {'':>10} {'':>10} {'':>10} {'':>10} || "
      f"{'':>10} {'':>10} || {check_total:10.2f} || ")


[Settlement Decomposition @ t=14, s=19]

  i |       x·DA     +y⁺·RT     -y⁻·PN        EXᵢ ||      given   received        INᵢ ||    EXᵢ+INᵢ ||     DISAGG
---------------------------------------------------------------------------------------------------------------------------
  0 |   36495.46       0.00  -35784.51     710.95 ||       0.00  -15764.14  -15764.14 ||  -15053.19 ||   13745.04 (-28798.2)
  1 |   60667.78    5115.51      -0.00   65783.29 ||   11612.19      -0.00   11612.19 ||   77395.48 ||   59479.07 (17916.4)
  2 |   14377.00   10648.62      -0.00   25025.62 ||   24172.32      -0.00   24172.32 ||   49197.94 ||   26226.17 (22971.8)
---------------------------------------------------------------------------------------------------------------------------
  Σ |                                    91519.86 ||                         20020.37 ||  111540.23 ||   99450.28

[Settlement Decomposition @ t=14, s=19]

  i |       x·DA     +y⁺·RT     -y⁻·PN        EXᵢ ||      given   re

In [58]:
internal_price = np.zeros((T, S))

for t, s in product(range(T), range(S)):
    internal_price[t, s] = P_PN[t] - P_RT[t, s]

for t in range(14, 20):
    print(f"[시간 {t}]")
    for s in range(9, 10):
        print(f"시나리오 {s}: {internal_price[t, s]} ( = {P_PN[t]} - {P_RT[t, s]} )")
        for i in range(I):
            print(f"[{i}] given {sum(d[i, j, t, s].X for j in range(I))} / received {sum(d[j, i, t, s].X for j in range(I))} / sold {ep[i, t, s].X} / penalty {em[i, t, s].X}")

[시간 14]
시나리오 9: 86.21731403403194 ( = 236.9835 - 150.76618596596805 )
[0] given 47.0 / received 0.0 / sold 0.0 / penalty 0.0
[1] given 0.0 / received 220.0 / sold 0.0 / penalty 0.0
[2] given 173.0 / received 0.0 / sold 0.0 / penalty 0.0
[시간 15]
시나리오 9: -81.02770045209905 ( = 256.269 - 337.29670045209906 )
[0] given 0.0 / received 0.0 / sold 81.0 / penalty 0.0
[1] given 0.0 / received 0.0 / sold 197.0 / penalty 0.0
[2] given 0.0 / received 0.0 / sold 91.0 / penalty 0.0
[시간 16]
시나리오 9: 89.99226011015537 ( = 267.0135 - 177.02123988984465 )
[0] given 0.0 / received 21.0 / sold 0.0 / penalty 0.0
[1] given 59.0 / received 0.0 / sold 160.0 / penalty 0.0
[2] given 0.0 / received 38.0 / sold 0.0 / penalty 0.0
[시간 17]
시나리오 9: 16.51007126928471 ( = 258.0825 - 241.57242873071527 )
[0] given 0.0 / received 0.0 / sold 0.0 / penalty 0.0
[1] given 0.0 / received 0.0 / sold 30.0 / penalty 0.0
[2] given 0.0 / received 0.0 / sold 123.0 / penalty 0.0
[시간 18]
시나리오 9: 52.03245600038318 ( = 216.606 - 164.573